Написать приложение, которое собирает основные новости с сайта на выбор lenta.ru, news.mail.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:
название источника;
наименование новости;
ссылку на новость;
дата публикации.
Сложить собранные данные в БД
Минимум один сайт, максимум - все три

In [1]:
from lxml import html
from pprint import pprint
import requests
import time

import numpy as np
import pandas as pd

from datetime import datetime
today = str(datetime.now().year) + '-' + str(datetime.now().month) + '-' + str(datetime.now().day) 

In [ ]:
def remove_xa0(list_):
  return [i.replace('\xa0', ' ') for i in list_]
  
def url_clean(list_, url = 'https://lenta.ru'):
  return [url + i if i.split('/')[0] != 'https:' else i for i in list_]

# вытаскиваем дату из url. Пhоблема в том что просто цифры не вытащишь-кроме даты URL могут содержать другие числа, а разбивать сплитом универсальсно для всех URL не получится
# так каr кроме lenta/ru есть moclenta и motor, у которых нет в URL указание на дату, и которым я присвоил значение сегоднешнего дня
def data_take(list_):
  return [i.split('/')[-5:-2] if (i).split('//')[-1].split('/')[0] == 'lenta.ru' else today.split('-') for i in list_]

def sourse(list_):
  return [i.split('/')[2] for i in list_]

# Lenta.ru

In [75]:
URL = 'https://lenta.ru/'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36'}

request_ = requests.get(URL, headers = header)
dom = html.fromstring(request_.text)


In [ ]:
# # На главной странице ленты видно что новости разбиты на некоторые блоки (которые к сожалению не совпадают с section):
# 1. Верхний левый блок
# 2. Блок "Главные новости"
# 3. Блок 'H3'
# 4. Foot - в самом низу новости разбитые по категориям: Россия, Мир, Бывший СССР итд. Если предыдущие блоки между собой никак не пересекаются то этот блок содержит как новые новости 
# так и уже попавшие

# Нам нужны текст и url. А из url можно достать источник и дату

## Foot

In [80]:
foot_for_url = '//div[contains(@class, "b-tabloid__row")]/..//a[contains(@href, "/rubrics")]/../following-sibling::*/a/@href'
get_foot_url = dom.xpath(foot_for_url)

foot_for_title = '//div[contains(@class, "b-tabloid__row")]/..//a[contains(@href, "/rubrics")]/../following-sibling::*/a/text() | \
                 //div[contains(@class, "b-tabloid__row")]/..//a[contains(@href, "/rubrics")]/../following-sibling::*/a/span/text()'
get_foot_title = dom.xpath(foot_for_title)

len(get_foot_title) == len(get_foot_url) 


True

## Главные новости

In [82]:
title_mn = '//div[contains(*, "Главные новости")]/div/a/text()'
url_mn = '//div[contains(*, "Главные новости")]/div/a/@href'

title_mn_get = dom.xpath(title_mn)
url_mn_get = dom.xpath(url_mn) 

len(title_mn_get) == len(url_mn_get)

True

## under H2 title

In [83]:
underH2_url = '//a/time/../@href'
get_url_h2 = dom.xpath(underH2_url)

underH2_title = '//a/time/../text()'
get_text_h2 = dom.xpath(underH2_title)  

len(get_url_h2) == len(get_text_h2)

True

## H3 title

In [84]:
get_url_h3  = dom.xpath('//a/h3/../@href')
get_text_h3 =  dom.xpath('//a/h3/text()')

# print(get_url_h3)
# print(get_text_h3)
len(get_url_h3) == len(get_text_h3)

True

## Суммируем все списки и получаем таблицу

In [85]:
url_all = get_foot_url + url_mn_get + get_url_h2 + get_url_h3
url_all = url_clean(url_all)
title_all = (get_foot_title + title_mn_get + get_text_h2 + get_text_h3)
data_all = data_take(url_all)

final = pd.DataFrame({'sourse': sourse(url_all), 'title': title_all, 'link': url_all})
final = pd.concat([final, pd.DataFrame(data_all, columns=['Year', 'Month', 'Day'])], axis=1)
final[['Year', 'Month', 'Day']] = final[['Year', 'Month', 'Day']].apply(pd.to_numeric)
final = final.drop_duplicates(subset='link').reset_index()
final

,index,sourse,title,link,Year,Month,Day
0,0,lenta.ru,Путин обсудил с премьером Пакистана ситуацию в...,https://lenta.ru/news/2021/08/25/pakistan/,2021,8,25
1,1,lenta.ru,МВД изменит правила проверки водителей на сост...,https://lenta.ru/news/2021/08/25/voditeli/,2021,8,25
2,2,lenta.ru,«Стало больше разрывов легких»,https://lenta.ru/articles/2021/08/26/covdelta/,2021,8,26
3,3,moslenta.ru,Охранники ТЦ не пустили россиянина на парковку...,https://moslenta.ru/news/okhranniki-tc-ne-pust...,2021,8,26
4,4,lenta.ru,«Не хочу больше этого зоопарка»,https://lenta.ru/articles/2021/08/23/externat/,2021,8,23
...,...,...,...,...,...,...,...
154,161,lenta.ru,Меньшее из зол.,https://lenta.ru/extlink/2021/08/23/bat/,2021,8,23
155,163,lenta.ru,Необычный список покупок для мужчины вызвал сп...,https://lenta.ru/news/2021/08/25/pokupki/,2021,8,25
156,165,lenta.ru,Россиянин сравнил поддельные кеды Gucci с ориг...,https://lenta.ru/news/2021/08/25/poddelka/,2021,8,25
157,167,moslenta.ru,Куклачев назвал фейком видео с котом и московс...,https://moslenta.ru/news/kuklachev-nazval-feik...,2021,8,26


# news.mail

In [86]:
URL = 'https://news.mail.ru/'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.116 Safari/537.36'}

request_ = requests.get(URL, headers = header)
dom = html.fromstring(request_.text)

In [113]:
def get(titles, urls, sourses, datatimes):
  for title, url, sourse, datatime in zip([titles], [urls], [sourses], [datatimes]):

    get_title    = (dom.xpath(title))
    get_url      = (dom.xpath(url))

    if sourse == 'none':
      get_sourse   = ['mail.ru'] * len(get_title)
    else:
      get_sourse   = (dom.xpath(sourse))

    if datatime == 'none':
      get_datatime = [today]*len(get_title)
    else:
      get_datatime = (dom.xpath(datatime))
    
    print(get_title, get_url, get_sourse, get_datatime)

    if len(get_title) == len(get_url) == len(get_sourse) == len(get_datatime):
      print('Длины совпали')
    else:
      print('Ошибка')
    return get_title, get_url, get_sourse, get_datatime

In [ ]:
# В отличии от ленты здесь можно воспользоваться текстовыми указателями, которые в отличие от ленты имею осмысленный и фиксированный вид
# Выделяется 4 блока 

In [114]:
# 6 картинок посредине
# только тут присутствует источник и дата, во всех остальных пропишем mail.ru и сегодняшнее число
sourses = '//span[contains(@class, "newsitem__param")][2]/text()'
datatimes = '//span[contains(@class, "newsitem__param")][1]/@datetime'
urls = '//span[contains(@class, "newsitem__title-inner")]/../@href'
titles = '//span[contains(@class, "newsitem__title-inner")]/text()'

titles1, urls1, sourses1, datatimes1 = get(titles, urls, sourses, datatimes)

['The Telegraph: Британия готовится прививать школьников от COVID-19 без разрешения родителей', 'Медленную вакцинацию бедных стран оценили в $2,3 трлн потерь в будущем', 'Нейтрализующая способность антител после «Спутника V» растет со временем', 'В Кабуле разграбили российский вертолет'] ['https://news.mail.ru/politics/47664099/', 'https://news.mail.ru/economics/47662731/', 'https://news.mail.ru/society/47663928/', 'https://news.mail.ru/incident/47657478/'] ['Коммерсантъ', 'РБК', 'ТАСС Наука', 'РБК'] ['2021-08-26T07:37:03+03:00', '2021-08-25T21:40:09+03:00', '2021-08-26T01:51:51+03:00', '2021-08-25T18:33:56+03:00']
Длины совпали


In [115]:
# 5 картинок сверху + 6 новостей ниже, причем возможно дублирование
# Тут нет указание на дату и источник видимо тут наиболее актуальные новости, присваиваем им значение сегодняшнего дня
urls = '//a[contains(@class, "top")]/@href   |  //span[contains(@class, "photo__title photo__title")]/../../@href | //a[contains(@class, "list__text")]/@href'
titles = '//span[contains(@class, "photo__title photo__title")]/text() | //a[contains(@class, "list__text")]/text()'
datatimes = 'none'
sourses = 'none'

titles2, urls2, sourses2, datatimes2 = get(titles, urls, sourses, datatimes)


['ЕС объяснил задержку в\xa0признании сертификатов о\xa0вакцинации с\xa0РФ\r\n', 'Вузы объявили дополнительный набор из-за нехватки студентов', 'Украинцы отказались от эвакуации из\xa0Афганистана на\xa0самолетах РФ', 'Минтруд изменит правила начисления соцвыплат', 'Младенец с обложки группы Nirvana подал в суд на группу', 'Минтруд изменит правила начисления соцвыплат', 'Младенец с обложки группы Nirvana подал в суд на группу', 'Кремль не\xa0приемлет позицию Турции по\xa0возвращению Крыма Украине', 'Исследование: эффект от\xa0вакцинации может длиться недолго', 'Назван самый безопасный город мира', 'День в\xa0истории: открытие Берингова пролива и битва за Днепр', 'Американский пляж усеяли сотни «песчаных долларов» (видео)', 'Талибы призвали жительниц Афганистана не выходить из дома'] ['https://news.mail.ru/politics/47664440/', 'https://news.mail.ru/society/47664063/', 'https://news.mail.ru/incident/47661828/', 'https://news.mail.ru/society/47664409/', 'https://news.mail.ru/society/476519

In [116]:
# 16 новостей под 6 картинками с указанными источниками
# Тоже никаких указаний на дату

titles = '//span/a/span[contains(@class, "link__text")]/text()'
urls = '//span/a/span[contains(@class, "link__text")]/../@href'
datatimes = 'none'
sourses = 'none'

titles3, urls3, sourses3, datatimes3 = get(titles, urls, sourses, datatimes)

['Бывший губернатор Нью-Йорка занижал смертность от коронавируса', 'Китай призвал ВОЗ проверить лаборатории США для поиска истоков пандемии', 'Байден заявил, что Россия знает, кто стоит за кибератаками', 'В ПФР разъяснили порядок назначения выплаты в 10 000 рублей', 'Пандемия коронавируса не спасла планету от загрязнения', '«Северный поток-2» проиграл спор с Европой', 'Эксперты ВОЗ заявили, что поиск истоков SARS-CoV-2 становится все труднее', 'В ВОЗ заявили о стабилизации в мире числа случаев заражения и смерти из-за коронавируса', 'Фото дня', 'СК завел дело о взятках биткоинами за диссертации в МПГУ', 'В России впервые с 17 августа число умерших из-за ковида за сутки превысило 800', 'Сотрудники МЧС вырвались из огненного кольца в Мордовии'] ['https://news.mail.ru/politics/47664022/', 'https://news.mail.ru/politics/47664824/', 'https://news.mail.ru/politics/47663906/', 'https://news.mail.ru/economics/47659693/', 'https://news.mail.ru/economics/47659437/', 'https://news.mail.ru/economi

In [117]:
# Новости в самом низу, справочного характера

titles = '//span[contains(@class, "collections__title")]/text()'
urls = '//span[contains(@class, "collections__title")]/../@href'
datatimes = 'none'
sourses = 'none'
titles4, urls4, sourses4, datatimes4 = (get(titles, urls, sourses, datatimes))

['«Полнейшая чепуха». Почему сдавать тесты на\xa0антитела к\xa0ковиду не\xa0имеет смысла', 'Как будут принимать туристов на курортах Кубани. Главные правила', 'Когда нужно ревакцинироваться и\xa0какой вакциной? Главные вопросы', '«Дельта плюс» вызывает все больше опасений. Что мы\xa0знаем об\xa0этом штамме коронавируса?', 'Пропускной режим: кто и как сможет попасть в московские рестораны с 28 июня', 'Какие ограничения вводят регионы из-за новой вспышки COVID-19. Главное', 'Индийский штамм коронавируса. Что нам о нем известно?', 'Вакцина «КовиВак» поступила в\xa0регионы. Чем она отличается от\xa0других', 'Ответы на главные вопросы о Петербургском международном экономическом форуме (ПМЭФ)', 'Новое обострение конфликта в секторе Газа. Главное', 'Новые санкции США против российского госдолга. Разбираемся, что это значит', '«В этом есть даже положительный эффект»: как новые санкции США отразятся на российской экономике', 'Когда отмечают День бухгалтера в России', 'Что такое эквайринг просты

## Итоговая таблица по mail.ru

In [124]:
final_mail = pd.DataFrame({'Title': titles1 + titles2 + titles3 + titles4,  
"Link": url_clean(urls1 + urls2 + urls3 + urls4, url = 'https://news.mail.ru'), 
'Sources': sourses1 + sourses2 + sourses3 + sourses4,
"Data": datatimes1 + datatimes2 + datatimes3 + datatimes4})
final_mail = final_mail.drop_duplicates(subset='Link').reset_index()
final_mail

,index,Title,Link,Sources,Data
0,0,The Telegraph: Британия готовится прививать шк...,https://news.mail.ru/politics/47664099/,Коммерсантъ,2021-08-26T07:37:03+03:00
1,1,Медленную вакцинацию бедных стран оценили в $2...,https://news.mail.ru/economics/47662731/,РБК,2021-08-25T21:40:09+03:00
2,2,Нейтрализующая способность антител после «Спут...,https://news.mail.ru/society/47663928/,ТАСС Наука,2021-08-26T01:51:51+03:00
3,3,В Кабуле разграбили российский вертолет,https://news.mail.ru/incident/47657478/,РБК,2021-08-25T18:33:56+03:00
4,4,ЕС объяснил задержку в признании сертификатов ...,https://news.mail.ru/politics/47664440/,mail.ru,2021-8-26
...,...,...,...,...,...
58,60,"Тест: Угадайте, на каком языке так признаются ...",https://news.mail.ru/society/47030263/,mail.ru,2021-8-26
59,61,Тест: Разбираетесь ли вы в денежных купюрах ра...,https://news.mail.ru/society/47000823/,mail.ru,2021-8-26
60,62,Тест: Насколько хорошо вы знаете историю моды?,https://news.mail.ru/society/46993546/,mail.ru,2021-8-26
61,63,Тест: Угадайте экзотический фрукт по фотографии,https://news.mail.ru/society/46903267/,mail.ru,2021-8-26
